In [11]:
import numpy as np
import pandas as pd
import pyhere
from datetime import datetime

In [12]:
dir_data_raw = pyhere.here().resolve().joinpath("data", "raw")
dir_data_interim = pyhere.here().resolve().joinpath("data", "interim")


In [13]:
csv_power_plants = pd.read_csv(dir_data_interim/"power_plants_with_generation_not_nan.csv", index_col=[0])
csv_power_plants

,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,other_fuel3,commissioning_year,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,generation_gwh_2017,generation_gwh_2018,generation_gwh_2019,generation_data_source,solar_wind_or_not
0,1.9,40.2003,-74.5761,Solar,NaN,NaN,NaN,2012.000000,2019.0,2.41461,2.35000,2.430000,2.492000,2.27600,2.207000,2.349000,U.S. Energy Information Administration,solar_wind
1,1.3,33.7943,-118.2414,Solar,NaN,NaN,NaN,2011.000000,2019.0,1.52800,2.14900,1.515000,1.592000,1.66000,2.139000,1.781010,U.S. Energy Information Administration,solar_wind
2,3.8,40.5358,-74.3913,Solar,NaN,NaN,NaN,2011.000000,2019.0,5.03600,4.52400,4.802000,5.051000,4.81900,4.626000,5.010000,U.S. Energy Information Administration,solar_wind
3,4.2,41.9084,-89.0466,Gas,NaN,NaN,NaN,2000.000000,2019.0,0.21500,0.17800,0.271000,0.306000,0.26400,0.272000,0.024000,U.S. Energy Information Administration,not
4,1.9,40.5161,-74.3400,Solar,NaN,NaN,NaN,2012.000000,2019.0,2.74100,2.58400,2.526000,2.659000,2.54700,2.386000,2.659000,U.S. Energy Information Administration,solar_wind
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6025,596.7,42.4776,-87.8950,Gas,Oil,NaN,NaN,2002.333333,2019.0,0.00000,63.65600,132.434005,435.493999,0.00000,522.427997,597.390002,U.S. Energy Information Administration,not
6026,7.0,42.4803,-87.8861,Waste,NaN,NaN,NaN,2003.000000,2019.0,43.59200,49.13400,46.699000,42.480000,38.66900,30.495000,22.647000,U.S. Energy Information Administration,not
6027,18.0,38.2803,-85.7023,Gas,NaN,NaN,NaN,1969.000000,2019.0,0.20300,0.07800,1.058000,0.062000,0.01500,-0.073000,-0.045000,U.S. Energy Information Administration,not
6028,3.4,42.8869,-76.9683,Wind,NaN,NaN,NaN,2012.000000,2019.0,3.67100,2.46015,2.489000,1.670000,1.23349,2.630000,1.211000,U.S. Energy Information Administration,solar_wind


In [172]:
sample_lat_lon = csv_power_plants[["latitude", "longitude"]].head(2)


In [173]:
list_of_tuples = list(sample_lat_lon.to_records(index=False))
list_of_tuples


[(32.322, 65.119), (31.67, 65.795)]

# LOOP SINGLE POINT

In [175]:
'''
*Version: 2.0 Published: 2021/03/09* Source: [NASA POWER](https://power.larc.nasa.gov/)
POWER API Multi-Point Download
This is an overview of the process to request data from multiple data points from the POWER API.
'''
# TQV                   MERRA-2 Total Column Precipitable Water (kg m-2) 
# WS10M                 MERRA-2 Wind Speed at 10 Meters (m/s) 
# CLRSKY_SFC_SW_DNI     CERES SYN1deg Clear Sky Surface Shortwave Downward Direct Normal Irradiance (kW-hr/m^2/day) 


import os, json, requests
from io import StringIO
import certifi
import urllib3
datetime_object = datetime.now()

locations = list_of_tuples
NORTH_HEMISPHERE_MONTHS_SEASONS = dict()
SOUTH_HEMISPHERE_MONTHS_SEASONS = dict()
MONTHS_OF_YEAR = np.array(["JAN", "FEB", "MAR", "APR", "MAY", "JUN", "JUL", "AUG", "SEP", "OCT", "NOV", "DEC"])
NORTH_HEMISPHERE_MONTHS_SEASONS["autumn"] = np.array(["OCT", "NOV", "DEC"])
NORTH_HEMISPHERE_MONTHS_SEASONS["winter"] = np.array(["JAN", "FEB", "MAR"])
NORTH_HEMISPHERE_MONTHS_SEASONS["spring"] = np.array(["APR", "MAY", "JUN"])
NORTH_HEMISPHERE_MONTHS_SEASONS["summer"] = np.array(["JUL", "AUG", "SEP"])
SOUTH_HEMISPHERE_MONTHS_SEASONS["spring"] = np.array(["OCT", "NOV", "DEC"])
SOUTH_HEMISPHERE_MONTHS_SEASONS["summer"] = np.array(["JAN", "FEB", "MAR"])
SOUTH_HEMISPHERE_MONTHS_SEASONS["autumn"] = np.array(["APR", "MAY", "JUN"])
SOUTH_HEMISPHERE_MONTHS_SEASONS["winter"] = np.array(["JUL", "AUG", "SEP"])

output = r""
base_url = r"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters=CLRSKY_SFC_SW_DNI,WS10M,TQV&community=RE&longitude={longitude}&latitude={latitude}&start=2000&end=2010&format=CSV&header=false"
df_response = pd.DataFrame()

http = urllib3.PoolManager(
    cert_reqs='CERT_REQUIRED',
    ca_certs=certifi.where()
)

for latitude, longitude in locations:
    api_request_url = base_url.format(longitude=longitude, latitude=latitude)

    response = http.request('GET', api_request_url, timeout=30.00).data.decode('utf-8')
    # response = requests.get(url=api_request_url, verify=False, timeout=30.00).content.decode('utf-8')

    # content = json.loads(response.content.decode('utf-8'))
    # filename = response.headers['content-disposition'].split('filename=')[1]
    filename_template = "{file_title}.csv"
    filename = filename_template.format(file_title = datetime_object.strftime("%Y-%m-%d_%H:%M:%S"))

    # print(pd.read_csv(StringIO(response)))
    df_response_aux = pd.read_csv(StringIO(response))
    df_response_aux["latitude"] = latitude
    df_response_aux["longitude"] = longitude

    if longitude > 0:
        hemisphere_months_seasons = NORTH_HEMISPHERE_MONTHS_SEASONS
    else:
        hemisphere_months_seasons = SOUTH_HEMISPHERE_MONTHS_SEASONS
    for index, element in hemisphere_months_seasons.items():
        df_response_aux[index]= df_response_aux[element].mean(axis=1)

    df_response_aux.drop(columns= MONTHS_OF_YEAR, inplace = True)

    df_response_aux = df_response_aux.pivot_table(index=["latitude", "longitude"], columns=["PARAMETER", "YEAR"])
    
    df_response_aux.columns = ["_".join(map(str, cols)) for cols in df_response_aux.columns.to_flat_index()]
    
    if(df_response.empty):
        
        df_response = df_response_aux.copy()
    else:
        df_response = pd.concat([df_response,df_response_aux])

# TRANSFORMING AND COMBINING DATA

df_response.to_csv(filename)
# with open(filename, 'w') as file_object:
#     file_object.write(response)
#     file_object.close()

# READING CSV TO CONTINUE TRANSFORMING THE DATA

In [152]:
df_nasa = pd.read_csv("File_Lat_32.322_Lon_65.119.csv", index_col=0)

df_nasa.head()

,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,SEP,OCT,NOV,DEC,ANN,latitude,longitude
0,TQV,2000,6.60,5.32,6.77,9.46,8.70,7.34,10.12,12.64,6.84,7.73,7.52,8.05,8.11,32.322,65.119
1,TQV,2001,5.29,5.48,7.41,10.46,8.55,8.52,12.42,8.80,7.08,7.19,6.21,9.66,8.11,32.322,65.119
2,TQV,2002,6.93,7.99,6.94,10.73,9.32,9.08,10.48,10.35,4.77,6.73,9.27,8.23,8.41,32.322,65.119
3,TQV,2003,7.63,8.88,10.16,11.14,8.79,9.09,18.09,11.17,7.34,6.50,6.51,7.68,9.43,32.322,65.119
4,TQV,2004,9.81,6.22,7.86,10.20,9.40,7.90,12.28,7.95,7.90,6.58,9.98,9.25,8.79,32.322,65.119


In [153]:

if float(df_nasa.loc[0, ["longitude"]]) > 0:
    hemisphere_months_seasons = NORTH_HEMISPHERE_MONTHS_SEASONS
else:
    hemisphere_months_seasons = SOUTH_HEMISPHERE_MONTHS_SEASONS
for index, element in hemisphere_months_seasons.items():
    df_nasa[index]= df_nasa[element].mean(axis=1)

In [154]:
df_nasa

,PARAMETER,YEAR,JAN,FEB,MAR,APR,MAY,JUN,JUL,AUG,...,OCT,NOV,DEC,ANN,latitude,longitude,autumn,winter,spring,summer
0,TQV,2000,6.60,5.32,6.77,9.46,8.70,7.34,10.12,12.64,...,7.73,7.52,8.05,8.11,32.322,65.119,7.766667,6.230000,8.500000,9.866667
1,TQV,2001,5.29,5.48,7.41,10.46,8.55,8.52,12.42,8.80,...,7.19,6.21,9.66,8.11,32.322,65.119,7.686667,6.060000,9.176667,9.433333
2,TQV,2002,6.93,7.99,6.94,10.73,9.32,9.08,10.48,10.35,...,6.73,9.27,8.23,8.41,32.322,65.119,8.076667,7.286667,9.710000,8.533333
3,TQV,2003,7.63,8.88,10.16,11.14,8.79,9.09,18.09,11.17,...,6.50,6.51,7.68,9.43,32.322,65.119,6.896667,8.890000,9.673333,12.200000
4,TQV,2004,9.81,6.22,7.86,10.20,9.40,7.90,12.28,7.95,...,6.58,9.98,9.25,8.79,32.322,65.119,8.603333,7.963333,9.166667,9.376667
5,TQV,2005,6.59,8.12,12.89,9.62,11.97,12.07,11.86,10.91,...,6.66,6.85,5.91,9.31,32.322,65.119,6.473333,9.200000,11.220000,10.316667
6,TQV,2006,7.33,10.73,8.18,9.34,11.16,8.12,12.55,14.98,...,9.79,10.60,7.83,9.77,32.322,65.119,9.406667,8.746667,9.540000,11.360000
7,TQV,2007,6.29,9.98,10.80,11.63,11.04,18.09,12.52,10.90,...,4.90,6.38,6.35,9.57,32.322,65.119,5.876667,9.023333,13.586667,9.836667
8,TQV,2008,6.13,5.44,7.77,10.05,9.27,12.15,13.54,12.37,...,8.17,7.33,7.77,9.09,32.322,65.119,7.756667,6.446667,10.490000,11.586667
9,TQV,2009,8.16,8.66,10.19,12.38,13.53,10.76,12.40,9.34,...,6.41,7.70,8.34,9.75,32.322,65.119,7.483333,9.003333,12.223333,10.276667


In [155]:
df_nasa.drop(columns= MONTHS_OF_YEAR, inplace = True)
df_nasa

,PARAMETER,YEAR,ANN,latitude,longitude,autumn,winter,spring,summer
0,TQV,2000,8.11,32.322,65.119,7.766667,6.230000,8.500000,9.866667
1,TQV,2001,8.11,32.322,65.119,7.686667,6.060000,9.176667,9.433333
2,TQV,2002,8.41,32.322,65.119,8.076667,7.286667,9.710000,8.533333
3,TQV,2003,9.43,32.322,65.119,6.896667,8.890000,9.673333,12.200000
4,TQV,2004,8.79,32.322,65.119,8.603333,7.963333,9.166667,9.376667
5,TQV,2005,9.31,32.322,65.119,6.473333,9.200000,11.220000,10.316667
6,TQV,2006,9.77,32.322,65.119,9.406667,8.746667,9.540000,11.360000
7,TQV,2007,9.57,32.322,65.119,5.876667,9.023333,13.586667,9.836667
8,TQV,2008,9.09,32.322,65.119,7.756667,6.446667,10.490000,11.586667
9,TQV,2009,9.75,32.322,65.119,7.483333,9.003333,12.223333,10.276667


In [156]:
df_nasa_pivot = df_nasa.pivot_table(index=["latitude", "longitude"], columns=["PARAMETER", "YEAR"])

In [157]:
df_nasa_pivot

ANN                                      \
PARAMETER          CLRSKY_SFC_SW_DNI                                       
YEAR                            2000  2001  2002  2003  2004  2005  2006   
latitude longitude                                                         
32.322   65.119               -999.0  8.06  8.14  7.89  8.24  8.37  7.94   

                                      ...    winter                      \
PARAMETER                             ...     WS10M                       
YEAR                2007  2008  2009  ...      2001      2002      2003   
latitude longitude                    ...                                 
32.322   65.119     8.14  7.82  7.72  ...  2.866667  2.893333  2.976667   

                                                                          
PARAMETER                                                                 
YEAR                    2004  2005  2006  2007  2008      2009      2010  
latitude longitude                                                        
32.322   65.119     2.956667  2.86  2.77  2.68  2.88  2.843333  2.963333  

[1 rows x 165 columns]

In [158]:
df_nasa_pivot.columns = ["_".join(map(str, a)) for a in df_nasa_pivot.columns.to_flat_index()]

In [159]:
df_nasa_pivot

,,ANN_CLRSKY_SFC_SW_DNI_2000,ANN_CLRSKY_SFC_SW_DNI_2001,ANN_CLRSKY_SFC_SW_DNI_2002,ANN_CLRSKY_SFC_SW_DNI_2003,ANN_CLRSKY_SFC_SW_DNI_2004,ANN_CLRSKY_SFC_SW_DNI_2005,ANN_CLRSKY_SFC_SW_DNI_2006,ANN_CLRSKY_SFC_SW_DNI_2007,ANN_CLRSKY_SFC_SW_DNI_2008,ANN_CLRSKY_SFC_SW_DNI_2009,...,winter_WS10M_2001,winter_WS10M_2002,winter_WS10M_2003,winter_WS10M_2004,winter_WS10M_2005,winter_WS10M_2006,winter_WS10M_2007,winter_WS10M_2008,winter_WS10M_2009,winter_WS10M_2010
latitude,longitude,,,,,,,,,,,,,,,,,,,,,
32.322,65.119,-999.0,8.06,8.14,7.89,8.24,8.37,7.94,8.14,7.82,7.72,...,2.866667,2.893333,2.976667,2.956667,2.86,2.77,2.68,2.88,2.843333,2.963333


In [129]:
first_row_csv_power_plants = csv_power_plants[['latitude', 'longitude', 'primary_fuel', 'capacity_mw']].head(1)

In [148]:
first_row_csv_power_plants.set_index(['latitude', 'longitude'], inplace=True)

In [150]:
pd.concat([first_row_csv_power_plants, df_nasa_pivot], axis=1)

,,primary_fuel,capacity_mw,ANN_CLRSKY_SFC_SW_DNI_2000,ANN_CLRSKY_SFC_SW_DNI_2001,ANN_CLRSKY_SFC_SW_DNI_2002,ANN_CLRSKY_SFC_SW_DNI_2003,ANN_CLRSKY_SFC_SW_DNI_2004,ANN_CLRSKY_SFC_SW_DNI_2005,ANN_CLRSKY_SFC_SW_DNI_2006,ANN_CLRSKY_SFC_SW_DNI_2007,...,winter_WS10M_2011,winter_WS10M_2012,winter_WS10M_2013,winter_WS10M_2014,winter_WS10M_2015,winter_WS10M_2016,winter_WS10M_2017,winter_WS10M_2018,winter_WS10M_2019,winter_WS10M_2020
latitude,longitude,,,,,,,,,,,,,,,,,,,,,
32.322,65.119,Hydro,33.0,-999.0,8.06,8.14,7.89,8.24,8.37,7.94,8.14,...,2.97,3.153333,2.8,2.666667,2.816667,2.906667,3.0,3.033333,2.943333,2.82
